In [6]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import tifffile

def resize_ld_images(em_dir, ld_dir, output_dir):
    """
    For each file in ld_dir, finds the same filename in em_dir,
    pads or crops the LD image to match the EM image shape (bottom/right),
    and writes the resized LD image to output_dir.  If an EM file has no
    corresponding LD mask, creates a black mask of the same size.
    Logs original and new shapes in a DataFrame.
    
    Parameters
    ----------
    em_dir : str or Path
        Path to folder of EM images.
    ld_dir : str or Path
        Path to folder of LD masks.
    output_dir : str or Path
        Path to folder where resized LD masks should be saved.
    
    Returns
    -------
    pandas.DataFrame
        Columns: filename, em_shape, ld_shape, new_ld_shape
    """
    em_dir = Path(em_dir)
    ld_dir = Path(ld_dir)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    records = []
    processed = set()
    
    # First, process existing LD files
    for ld_path in ld_dir.iterdir():
        if not ld_path.is_file():
            continue
        em_path = em_dir / ld_path.name
        if not em_path.exists():
            print(f"⚠️  EM file not found for {ld_path.name}, skipping LD.")
            continue
        
        # load images
        em_img = tifffile.imread(str(em_path))
        ld_img = tifffile.imread(str(ld_path))
        em_h, em_w = em_img.shape[:2]
        ld_h, ld_w = ld_img.shape[:2]
        new = ld_img
        
        # crop if too large
        if ld_h > em_h:
            new = new[:em_h, ...]
        if ld_w > em_w:
            new = new[:, :em_w, ...]
        
        # pad if too small
        pad_h = max(em_h - new.shape[0], 0)
        pad_w = max(em_w - new.shape[1], 0)
        if pad_h or pad_w:
            pad_width = [(0, pad_h), (0, pad_w)]
            for _ in range(new.ndim - 2):
                pad_width.append((0, 0))
            new = np.pad(new, pad_width, mode='constant', constant_values=0)
        
        new = (new > 0).astype(np.uint8) * 255
        
        # sanity-check
        assert new.shape[:2] == (em_h, em_w), \
            f"{ld_path.name} → final shape {new.shape[:2]} != EM shape {(em_h, em_w)}"
        
        # write out
        out_path = output_dir / ld_path.name
        tifffile.imwrite(str(out_path), new)
        processed.add(ld_path.name)
        
        # record
        records.append({
            "filename": ld_path.name,
            "em_shape": (em_h, em_w),
            "ld_shape": (ld_h, ld_w),
            "new_ld_shape": new.shape[:2]
        })
    
    # Next, handle EM files without LD masks
    for em_path in em_dir.iterdir():
        if not em_path.is_file():
            continue
        if em_path.name in processed:
            continue
        
        # load EM to get shape
        em_img = tifffile.imread(str(em_path))
        em_h, em_w = em_img.shape[:2]
        
        # create black mask
        black = np.zeros((em_h, em_w), dtype=np.uint8)
        out_path = output_dir / em_path.name
        tifffile.imwrite(str(out_path), black)
        
        # record
        records.append({
            "filename": em_path.name,
            "em_shape": (em_h, em_w),
            "ld_shape": (0, 0),
            "new_ld_shape": (em_h, em_w)
        })
    
    return pd.DataFrame(records)


ems = "/Volumes/Chris_SSD/file_shuttle/EM-lds"
lds = "/Volumes/Chris_SSD/file_shuttle/masks(lipid droplets)"
output_dir = "/Volumes/Chris_SSD/file_shuttle/lds-output"

resize_ld_images(ems, lds, output_dir)

,filename,em_shape,ld_shape,new_ld_shape
0,25-0073_5nm_Region12_bsd-1.tif,"(1294, 1294)","(1158, 953)","(1294, 1294)"
1,25-0073_5nm_Region12_bsd-2.tif,"(1294, 1294)","(858, 1249)","(1294, 1294)"
2,25-0073_5nm_Region12_bsd-3.tif,"(1294, 1294)","(908, 724)","(1294, 1294)"
3,25-0075_5nm_Region11_bsd-1.tif,"(1038, 1038)","(77, 882)","(1038, 1038)"
4,25-0075_5nm_Region11_bsd-2.tif,"(1038, 1038)","(976, 461)","(1038, 1038)"
5,25-0075_5nm_Region11_bsd-3.tif,"(1038, 1038)","(546, 870)","(1038, 1038)"
6,25-0075_5nm_Region11_bsd-4.tif,"(1038, 1038)","(1020, 1043)","(1038, 1038)"
7,25-0077_5nm_R11_bsd-1.tif,"(1057, 1057)","(949, 820)","(1057, 1057)"
8,25-0077_5nm_R11_bsd-3.tif,"(1057, 1057)","(250, 288)","(1057, 1057)"
9,25-0077_5nm_R11_bsd-4.tif,"(1096, 1096)","(1029, 296)","(1096, 1096)"


In [4]:
import tifffile
import os
import numpy as np

# Load the files in a given folder, and output 8bit versions of them (they are currently 32floats)
def load_and_convert_to_8bit(folder_path):
    # output is sibling folder of input, named "ld_8bit"
    output_path = "/Volumes/Chris_SSD/lds_20250625_8bit"
    os.makedirs(output_path, exist_ok=True)
    for file in os.listdir(folder_path):
        if file.endswith(".tif"):
            img = tifffile.imread(os.path.join(folder_path, file))
            img = (img * 255).astype(np.uint8)
            tifffile.imwrite(os.path.join(folder_path, file), img)


input_folder = "/Volumes/Chris_SSD/lds_20250625"
load_and_convert_to_8bit(input_folder)



/var/folders/v5/v1r03h0s3f50cyrz1tz2sfqc0000gn/T/ipykernel_68040/3341495707.py:13: RuntimeWarning: invalid value encountered in cast
  img = (img * 255).astype(np.uint8)


In [ ]:
import os 
import numpy as np
import tifffile

def _resize_to(img: np.ndarray, target_shape: tuple[int,int]) -> np.ndarray:
    """
    Pads with zeros or crops on bottom/right so that
    `img.shape == target_shape`.
    """
    h_tgt, w_tgt = target_shape
    h, w = img.shape
    if img.shape == target_shape:
        return img

    # crop if too big
    h_crop = min(h, h_tgt)
    w_crop = min(w, w_tgt)
    img = img[:h_crop, :w_crop]

    # pad if too small
    pad_h = h_tgt - h_crop
    pad_w = w_tgt - w_crop
    if pad_h > 0 or pad_w > 0:
        img = np.pad(
            img,
            ((0, pad_h), (0, pad_w)),
            mode="constant",
            constant_values=0
        )
    return img

for file in os.listdir("/Volumes/Chris_SSD/file_shuttle/glycogen_masks"):
    if file.endswith(".tif"):
        img = tifffile.imread(os.path.join("/Volumes/Chris_SSD/file_shuttle/glycogen_masks", file))
        fn = file.split("/")[-1]
        original_img = tifffile.imread(os.path.join("/Volumes/Chris_SSD/file_shuttle/EM", fn))
        orignal_shape = original_img.shape
        img = _resize_to(img, orignal_shape)
        img = img > 0
        img = img.astype(np.uint8) * 255
        tifffile.imwrite(os.path.join("/Volumes/Chris_SSD/file_shuttle/glycogen_masks_resized", file), img)
        print(f"✓ resized {file} to {orignal_shape}")

25-0073_5nm_Region12_bsd-1.tif
✓ resized 25-0073_5nm_Region12_bsd-1.tif to (1294, 1294)
25-0073_5nm_Region12_bsd-2.tif
✓ resized 25-0073_5nm_Region12_bsd-2.tif to (1294, 1294)
25-0073_5nm_Region12_bsd-3.tif
✓ resized 25-0073_5nm_Region12_bsd-3.tif to (1294, 1294)
25-0075_5nm_Region11_bsd-1.tif
✓ resized 25-0075_5nm_Region11_bsd-1.tif to (1038, 1038)
25-0075_5nm_Region11_bsd-2.tif
✓ resized 25-0075_5nm_Region11_bsd-2.tif to (1038, 1038)
25-0075_5nm_Region11_bsd-3.tif
✓ resized 25-0075_5nm_Region11_bsd-3.tif to (1038, 1038)
25-0075_5nm_Region11_bsd-4.tif
✓ resized 25-0075_5nm_Region11_bsd-4.tif to (1038, 1038)
25-0077_5nm_R11_bsd-1.tif
✓ resized 25-0077_5nm_R11_bsd-1.tif to (1057, 1057)
25-0077_5nm_R11_bsd-2.tif
✓ resized 25-0077_5nm_R11_bsd-2.tif to (1057, 1057)
25-0077_5nm_R11_bsd-3.tif
✓ resized 25-0077_5nm_R11_bsd-3.tif to (1057, 1057)
25-0077_5nm_R11_bsd-4.tif
✓ resized 25-0077_5nm_R11_bsd-4.tif to (1096, 1096)
